<!-- TODO(ccy): split this notebook into (1) a beginner friendly notebook that
     avoids directly calling into TFX libraries and (2) an advanced notebook
     or notebooks that delve more deeply into each component, along with their
     underlying libraries. -->
# TFX Iterative Development Example
This notebook demonstrates how to use Jupyter notebooks for TFX iterative development.  Here, we walk through the Chicago Taxi example in an interactive Jupyter notebook.

Note: this notebook along with its associated APIs are **experimental** and are
in active development.  Major changes in functionality, behavior and
presentation are expected.

## Setup
First, download data, import modules and set up paths.

### Import packages
We import necessary packages, including standard TFX component classes.

In [0]:
import os
import tempfile
import urllib

import tensorflow as tf
import tfx
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.interactive.interactive_context import check_ipython
from tfx.orchestration.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import csv_input

### Load custom extensions.
Use cell magic `%%skip_for_export` to denote cells to skip during export to pipeline.

In [0]:
%load_ext tfx.orchestration.interactive.notebook_extensions.skip

### Set up pipeline paths

In [0]:
# Set up paths.

if check_ipython():
  _tfx_root = tfx.__path__[0]
  _taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')
  # Path which can be listened to by the model server.  Pusher will output the
  # trained model here.
  _serving_model_dir = os.path.join(
      tempfile.mkdtemp(), 'serving_model/taxi_simple')
else:
  # Paths for exported pipeline.
  _tfx_root = os.path.join(os.environ['HOME'], 'tfx')
  _taxi_root = os.path.join(os.environ['HOME'], 'taxi')
  _serving_model_dir = os.path.join(_taxi_root, 'serving_model')

# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_taxi_module_file = os.path.join(_taxi_root, 'taxi_utils.py')

### Download example data
We download the sample dataset for use in our TFX pipeline.

In [0]:
# Download the example data.
if check_ipython():
  _data_root = tempfile.mkdtemp(prefix='tfx-data')
  DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
  with open(os.path.join(_data_root, 'data.csv'), 'wb') as f:
      contents = urllib.request.urlopen(DATA_PATH).read()
      f.write(contents)
else:
  _data_root = os.path.join(_taxi_root, 'data', 'simple')

## Create the InteractiveContext
We now create the interactive context.

In [0]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

## Run TFX components interactively
Next, we construct TFX components and run each one interactively using within the interactive session to obtain `ExecutionResult` objects.

### ExampleGen
`ExampleGen` brings data into the TFX pipeline.

In [0]:
# Use the packaged CSV input data.
examples = csv_input(_data_root)

# Brings data into the pipeline or otherwise joins/converts training data.
example_gen = CsvExampleGen(input_base=examples)
context.run(example_gen)

### StatisticsGen (using Tensorflow Data Validation)
`StatisticsGen` computes statistics for visualization and example validation. This uses the [Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.

#### Run TFDV statistics computation using the StatisticsGen component

In [0]:
# Computes statistics over data for visualization and example validation.
statistics_gen = StatisticsGen(
    input_data=example_gen.outputs['examples'])
context.run(statistics_gen)

#### Import TFDV and visualize the statistics result

In [0]:
%%skip_for_export

# Import TFDV and get the train statistics path.
import tensorflow_data_validation as tfdv
from tfx.types.artifact_utils import get_split_uri

artifact_list = statistics_gen.outputs['output'].get()
train_artifact_uri = get_split_uri(artifact_list, 'train')
train_stats_path = os.path.join(train_artifact_uri, 'stats_tfrecord')

In [0]:
%%skip_for_export

# Load statistics and visualize training data stats.

train_stats = tfdv.load_statistics(train_stats_path)
tfdv.visualize_statistics(train_stats)

### SchemaGen (using Tensorflow Data Validation)
`SchemaGen` generates a schema for your data based on computed statistics. This component also uses the [Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.

#### Run TFDV schema inference using the SchemaGen component

In [0]:
# Generates schema based on statistics files.
infer_schema = SchemaGen(
    stats=statistics_gen.outputs['output'],
    infer_feature_shape=False)
context.run(infer_schema)

#### Visualize the inferred schema

In [0]:
%%skip_for_export

# Get the schema path.
schema_dir = infer_schema.outputs['output'].get()[0].uri
schema_path = os.path.join(schema_dir, 'schema.pbtxt')

# Load and visualize the generated schema.
schema = tfdv.load_schema_text(schema_path)
tfdv.display_schema(schema)

### ExampleValidator
`ExampleValidator` performs anomaly detection based on computed statistics and your data schema.

#### Run TFDV data validation using the ExampleValidation component

In [0]:
# Performs anomaly detection based on statistics and data schema.
validate_stats = ExampleValidator(
    stats=statistics_gen.outputs['output'],
    schema=infer_schema.outputs['output'])
context.run(validate_stats)

#### Visualize the detected anomalies

In [0]:
%%skip_for_export

# Get the validation path.
validation_dir = validate_stats.outputs['output'].get()[0].uri
anomalies_path = os.path.join(validation_dir, 'anomalies.pbtxt')

# Load and visualize the anomalies.
anomalies = tfdv.load_anomalies_text(anomalies_path)
tfdv.display_anomalies(anomalies)

### Transform
`Transform` performs data transformations and feature engineering which are kept in sync for training and serving.

#### Run the Transform component

In [0]:
# Performs transformations and feature engineering in training and serving.
transform = Transform(
    input_data=example_gen.outputs['examples'],
    schema=infer_schema.outputs['output'],
    module_file=_taxi_module_file)
context.run(transform)

### Trainer
`Trainer` trains your custom model using TF-Learn.

In [0]:
# Uses user-provided Python function that implements a model using TF-Learn.
trainer = Trainer(
    module_file=_taxi_module_file,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=infer_schema.outputs['output'],
    transform_output=transform.outputs['transform_output'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))
context.run(trainer)

### Evaluator (using Tensorflow Model Analysis)
The `Evaluator` computes evaluation statistics over features of your model using [Tensorflow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started). In this section, we run TFMA in our TFX pipeline and then visualize the results to analyze the performance of our model.

#### Run TFMA using the Evaluator component

Here, we first define slicing specs for analyzing our data. Next, we run TFMA using these specs to generate results.

In [0]:
# An empty slice spec means the overall slice, that is, the whole dataset.
OVERALL_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec()

# Data can be sliced along a feature column
# In this case, data is sliced along feature column trip_start_hour.
FEATURE_COLUMN_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec(
    column_for_slicing=['trip_start_hour'])

# Data can be sliced by crossing feature columns
# In this case, slices are computed for trip_start_day x trip_start_month.
FEATURE_COLUMN_CROSS_SPEC = evaluator_pb2.SingleSlicingSpec(
    column_for_slicing=['trip_start_day', 'trip_start_month'])

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
    FEATURE_COLUMN_SLICE_SPEC,
    FEATURE_COLUMN_CROSS_SPEC,
]

In [0]:
# Use TFMA to compute a evaluation statistics over features of a model.
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model_exports=trainer.outputs['output'],
    feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(
        specs=ALL_SPECS
    ))
context.run(model_analyzer)

#### Get the TFMA output result path

In [0]:
PATH_TO_RESULT = model_analyzer.outputs['output'].get()[0].uri

#### Import TFMA and load the result

In [0]:
%%skip_for_export

import tensorflow_model_analysis as tfma

tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

#### Visualization: Slicing Metrics

To see the slices, either use the name of the column (by setting slicing_column) or provide a tfma.slicer.SingleSliceSpec (by setting slicing_spec). If neither is provided, an overall visualization will be displayed.

The default visualization is the **slice overview** when the number of slices is small. It shows the value of a metric for each slice, sorted by the another metric. It is also possible to set a threshold to filter out slices with smaller weights.

This view also supports the **metrics histogram** as an alternative visualization. It is also the default view when the number of slices is large. The results will be divided into buckets and the number of slices / total weights / both can be visualized. Slices with small weights can be filtered out by setting the threshold. Further filtering can be applied by dragging the grey band. To reset the range, double click the band. Filtering can be used to remove outliers in the visualization and the metrics table below.

In [0]:
%%skip_for_export

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='trip_start_hour')

In [0]:
%%skip_for_export

# Show metrics sliced by 'trip_start_day' x 'trip_start_month'.
tfma.view.render_slicing_metrics(
    tfma_result,
    slicing_spec=tfma.slicer.SingleSliceSpec(
        columns=['trip_start_day','trip_start_month']))

In [0]:
%%skip_for_export

# Show overall metrics.
tfma.view.render_slicing_metrics(tfma_result)

### ModelValidator
`ModelValidator` performs validation of your candidate model compared to a baseline.

In [0]:
# Performs quality validation of a candidate model (compared to a baseline).
model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['output'])
context.run(model_validator)

### Pusher
`Pusher` checks whether a model has passed validation, and if so, pushes the model to a file destination.

In [0]:
# Checks whether the model passed the validation steps and pushes the model
# to a file destination if check passed.
pusher = Pusher(
    model_export=trainer.outputs['output'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

## Export to pipeline

### Mount Google Drive (Colab).
Save a copy of your notebook in Google Drive if you haven't already, then run the following cell to mount drive.

In [0]:
%%skip_for_export

import sys

if 'google.colab' in sys.modules:
  # Colab.
  from google.colab import drive

  drive.mount('/content/drive')

### Set up notebook/export paths.

In [0]:
# TODO(USER).
notebook_filename = 'taxi_pipeline_interactive.ipynb'

# Colab example.
# TODO(USER): Update notebook directory path.
_notebook_filepath = os.path.join('/content/drive/My Drive/Colab Notebooks',
                                  notebook_filename)
_export_filepath_template = '/content/export_%s.py'

# Jupyter example.
# _notebook_filepath = os.path.join(os.getcwd(), notebook_filename)
# _export_filepath_template = os.path.join(os.getcwd(), 'export_%s.py')

### Save your notebook, then run the appropriate cell below to export a pipeline to run on Beam/Airflow/etc.

In [0]:
# Beam pipeline export template.

from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner


# Components to be part of the exported pipeline.
components = [
    example_gen, statistics_gen, infer_schema, validate_stats, transform,
    trainer, model_analyzer, model_validator, pusher
]

tf.logging.set_verbosity(tf.logging.INFO)

_pipeline_name = 'chicago_taxi_beam'
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
# Sqlite ML-metadata db path.
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

tfx_pipeline = pipeline.Pipeline(
    pipeline_name=_pipeline_name,
    pipeline_root=_pipeline_root,
    components=components,
    enable_cache=True,
    metadata_connection_config=(
        metadata.sqlite_metadata_connection_config(_metadata_path)),
    additional_pipeline_args={})

if not check_ipython():
  BeamDagRunner().run(tfx_pipeline)

export_filepath = _export_filepath_template % _pipeline_name
context.export_to_pipeline(notebook_filepath=_notebook_filepath,
                           export_filepath=export_filepath)

In [0]:
%%skip_for_export

# # Airflow pipeline export template.

# import datetime
# from tfx.orchestration.airflow.airflow_dag_runner import AirflowDagRunner

# # Components to be part of the exported pipeline.
# components = [
#     example_gen, statistics_gen, infer_schema, validate_stats, transform,
#     trainer, model_analyzer, model_validator, pusher
# ]

# tf.logging.set_verbosity(tf.logging.INFO)

# _pipeline_name = 'chicago_taxi_airflow'
# _pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
# # Sqlite ML-metadata db path.
# _metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
#                               'metadata.db')

# # Airflow-specific configs; these will be passed directly to airflow
# _airflow_config = {
#     'schedule_interval': None,
#     'start_date': datetime.datetime(2019, 1, 1),
# }

# tfx_pipeline = pipeline.Pipeline(
#     pipeline_name=_pipeline_name,
#     pipeline_root=_pipeline_root,
#     components=components,
#     enable_cache=True,
#     metadata_connection_config=(
#         metadata.sqlite_metadata_connection_config(_metadata_path)),
#     additional_pipeline_args={})

# if not check_ipython():
#   # 'DAG' below needs to be kept for Airflow to detect dag.
#   DAG = AirflowDagRunner(_airflow_config).run(tfx_pipeline)

# export_filepath = _export_filepath_template % _pipeline_name
# context.export_to_pipeline(
#     notebook_filepath=_notebook_filepath,
#     export_filepath=export_filepath)

### Download exported script from Colab.

In [0]:
%%skip_for_export

if 'google.colab' in sys.modules:
  from google.colab import files

  files.download(export_filepath)